# In deze analyse zullen we voor een fruitteler gaan kijken hoeveel de beste man goedkoper uit is met een dynamisch energiecontract versus zijn huidige contract.

Wat basis informatie over de desbetreffende potentiële klant:


Totaal Opgewekte Energie: 900.000 kWh
Zelf Verbruikte Energie: 450.000 kWh
Totaal Verbruik: Tussen 1.400.000 kWh en 1.500.000 kWh
Aansluitingen: 1 Grootverbruik (GV) en 2 Kleinverbruik (KV)
Huidig Energiecontract: Tot 31-12-2024 met van Helden Energie


we hebben een excel bestand aangeleverd gekregen met: bestand van de uurwaarde van vorige week met bijbehorende afname en welk volume schakelbaar is in tabel weergegeven & een csv met een lijn grafiek.

In [22]:
import pandas as pd
import openpyxl



In [23]:
df = pd.read_excel('Consumptie_kwartierdata_2023.xlsx', skiprows=[0])
df

,Datum-tijd tot,Elektriciteit Consumptie (kWh),Blindverbruik Consumptie (kVARh),Elektriciteit Consumptie (kWh).1,Blindverbruik Consumptie (kVARh).1,Elektriciteit Consumptie (kWh).2,Blindverbruik Consumptie (kVARh).2
0,2023-01-01 00:15:00,27.8,13.6,0,0.0,NaN,NaN
1,2023-01-01 00:30:00,30.3,14.0,0,0.0,NaN,NaN
2,2023-01-01 00:45:00,44.0,26.1,0,0.0,NaN,NaN
3,2023-01-01 01:00:00,38.5,24.8,0,0.0,NaN,NaN
4,2023-01-01 01:15:00,26.6,11.3,0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
35035,2023-12-31 23:00:00,20.5,7.5,0,0.0,0.0,0.0
35036,2023-12-31 23:15:00,25.9,8.8,0,0.0,0.0,0.0
35037,2023-12-31 23:30:00,22.1,7.4,0,0.0,0.0,0.0
35038,2023-12-31 23:45:00,20.6,7.5,0,0.0,0.0,0.0


prima dataset zo echter moeten we nog het een en ander schoonmaken voordat we verder gaan met calculaties
We gaan de NaN values omzetten naar het getal 0. Dit is mogelijk omdat er geen data beschikbaar is voor die records maar 0 voldoet als alternatief om een sommatie mogelijk te maken.

- eerste kijken we waar nan values zitten
- vervolgens vervangen we de nan values met 0

In [29]:
nan_df = df.isna()
df_clean = df.fillna(0)
df_clean

#factor to multiply the kVARH columns with is 0.8 so:
columns_to_multiply = ['Blindverbruik Consumptie (kVARh)', 'Blindverbruik Consumptie (kVARh).1','Blindverbruik Consumptie (kVARh).2']
factor = 0.8


TypeError: can't multiply sequence by non-int of type 'float'

In [27]:
Total_kVARH1 = df['Blindverbruik Consumptie (kVARh)'].sum()
Total_kVARH2 = df['Blindverbruik Consumptie (kVARh).1'].sum()
Total_kVARH3 = df['Blindverbruik Consumptie (kVARh).2'].sum()
print(Total_kVARH1)
print(Total_kVARH2)
print(Total_kVARH3)

Totaal_kVARH = (Total_kVARH1 + Total_kVARH2 + Total_kVARH3)
print(f'Totaal aan kvhrh: {Totaal_kVARH}')

695782.549
17.5
1131.7000000000003
Totaal aan kvhrh: 696931.749


#### we gaan een powerfactor van 0.8 gebruiken voor het omrekenen van kVARh naar kWh om een inschatting van kostenbesparing 

Optellen vhet verbruik 